# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_df = pd.read_csv('output.csv', encoding="ISO-8859-1")

output_df.head()


,Unnamed: 0,Date,Cities,Lat,Lon,Cloudiness(%),Max Temp,Wind Speed,Country,Humidity
0,0,01/26/21,yellowknife,62.4560,-114.3525,75,-11.002,4.12,KZ,64
1,1,01/26/21,new norfolk,-42.7826,147.0587,8,57.002,0.89,KZ,61
2,2,01/26/21,talnakh,69.4865,88.3972,100,-22.792,3.86,KZ,81
3,3,01/26/21,raudeberg,61.9875,5.1352,20,30.200,3.60,KZ,93
4,4,01/26/21,la ronge,55.1001,-105.2842,37,-16.600,2.06,KZ,69


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gmaps
gmaps.configure(api_key=g_key) 

# set up locations and humidity
locations = output_df[["Lat", "Lon"]]
humidity = output_df["Humidity"]

# plot heatmap
# use center and zoom_level so it's not so zoomed out
fig = gmaps.figure(center = (0,0), zoom_level=2)

# create the heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=400, point_radius=4)

# add layer
fig.add_layer(heat_layer)

# show figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Set parameters for ideal climate conditions and see the output
ideal_weather = output_df.loc[(output_df["Max Temp"]<75) &
                              (output_df["Max Temp"]>65) & 
                              (output_df["Wind Speed"]<10) & 
                              (output_df["Cloudiness(%)"]==0)]
ideal_weather.dropna()

,Unnamed: 0,Date,Cities,Lat,Lon,Cloudiness(%),Max Temp,Wind Speed,Country,Humidity
13,13,01/26/21,dakar,14.6937,-17.4441,0,69.800,3.09,KZ,64
112,112,01/26/21,salalah,17.0151,54.0924,0,68.000,1.03,KZ,68
122,122,01/26/21,gorele,41.0308,39.0031,0,66.200,5.14,KZ,22
201,201,01/26/21,praia,14.9215,-23.5087,0,73.400,5.66,KZ,49
306,306,01/26/21,le port,-20.9373,55.2919,0,73.400,2.57,KZ,88
330,330,01/26/21,sarh,9.1429,18.3923,0,71.600,1.56,KZ,60
344,344,01/26/21,rafai,4.9500,23.9167,0,73.004,1.56,KZ,42
346,346,01/26/21,sur,22.5667,59.5289,0,65.948,2.33,KZ,64
356,356,01/26/21,odienne,9.5100,-7.5692,0,74.966,1.13,KZ,20
426,426,01/26/21,adrar,20.5022,-10.0711,0,69.188,3.61,KZ,18


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = ideal_weather.dropna()

hotel_df["Hotel Name"] = ""
hotel_df

,Unnamed: 0,Date,Cities,Lat,Lon,Cloudiness(%),Max Temp,Wind Speed,Country,Humidity,Hotel Name
13,13,01/26/21,dakar,14.6937,-17.4441,0,69.800,3.09,KZ,64,
112,112,01/26/21,salalah,17.0151,54.0924,0,68.000,1.03,KZ,68,
122,122,01/26/21,gorele,41.0308,39.0031,0,66.200,5.14,KZ,22,
201,201,01/26/21,praia,14.9215,-23.5087,0,73.400,5.66,KZ,49,
306,306,01/26/21,le port,-20.9373,55.2919,0,73.400,2.57,KZ,88,
330,330,01/26/21,sarh,9.1429,18.3923,0,71.600,1.56,KZ,60,
344,344,01/26/21,rafai,4.9500,23.9167,0,73.004,1.56,KZ,42,
346,346,01/26/21,sur,22.5667,59.5289,0,65.948,2.33,KZ,64,
356,356,01/26/21,odienne,9.5100,-7.5692,0,74.966,1.13,KZ,20,
426,426,01/26/21,adrar,20.5022,-10.0711,0,69.188,3.61,KZ,18,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"radius": 5000, "type": "lodging", "key": g_key,}

# loop thorugh lon and lat
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lon = row["Lon"]

    # add location definition to params dict
    params['location'] = f"{lat},{lon}"

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = response["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("No Hotels Found")
        
    print("Hotel Found")
hotel_df

Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
No Hotels Found
Hotel Found
Hotel Found
Hotel Found
No Hotels Found
Hotel Found
Hotel Found
No Hotels Found
Hotel Found
No Hotels Found
Hotel Found


,Unnamed: 0,Date,Cities,Lat,Lon,Cloudiness(%),Max Temp,Wind Speed,Country,Humidity,Hotel Name
13,13,01/26/21,dakar,14.6937,-17.4441,0,69.800,3.09,KZ,64,Novotel Dakar
112,112,01/26/21,salalah,17.0151,54.0924,0,68.000,1.03,KZ,68,HAMDAN PLAZA HOTEL SALALAH
122,122,01/26/21,gorele,41.0308,39.0031,0,66.200,5.14,KZ,22,Görele Öğretmenevi
201,201,01/26/21,praia,14.9215,-23.5087,0,73.400,5.66,KZ,49,LT Aparthotel
306,306,01/26/21,le port,-20.9373,55.2919,0,73.400,2.57,KZ,88,Villa Maido
330,330,01/26/21,sarh,9.1429,18.3923,0,71.600,1.56,KZ,60,Hôtel des Chasses
344,344,01/26/21,rafai,4.9500,23.9167,0,73.004,1.56,KZ,42,
346,346,01/26/21,sur,22.5667,59.5289,0,65.948,2.33,KZ,64,Sur Plaza Hotel
356,356,01/26/21,odienne,9.5100,-7.5692,0,74.966,1.13,KZ,20,Oustaz Vame DIAKITE
426,426,01/26/21,adrar,20.5022,-10.0711,0,69.188,3.61,KZ,18,


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

KeyError: 'City'

In [10]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info) 

# Display figure
fig.add_layer(marker_layer)
fig

NameError: name 'hotel_info' is not defined